In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import tensorflow as tf
from tensorflow import keras

print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__)
    print(module.__version__)

sys.version_info(major=3, minor=7, micro=5, releaselevel='final', serial=0)
matplotlib
3.1.2
numpy
1.17.4
pandas
0.25.3
sklearn
0.22
tensorflow
2.0.0
tensorflow_core.keras
2.2.4-tf


## 1.Constant 

### 1.1.1 definition

In [2]:
t = tf.constant(value=[[1., 2., 3., 4], [5., 6., 7., 8.]], 
                dtype=tf.float32, 
                shape=(4, 2),
                name="tdata")
print(t)

tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]
 [7. 8.]], shape=(4, 2), dtype=float32)


### 1.1.2 indice

In [3]:
print(t[2:, :])

tf.Tensor(
[[5. 6.]
 [7. 8.]], shape=(2, 2), dtype=float32)


### 1.1.3 operation

In [4]:
t = tf.constant(value=[[1., 2., 3.], [4., 5., 6]])
print(t+10)
print(tf.square(t))
print(t @ tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


### 1.1.4 numpy conversion

In [5]:
t = tf.constant([1,2,3])
print(t.numpy())
print(np.square(t))
np_t = np.array([4, 5, 6])
print(tf.constant(np_t))

[1 2 3]
[1 4 9]
tf.Tensor([4 5 6], shape=(3,), dtype=int32)


### 1.1.5 scalers

In [6]:
t = tf.constant(2.7)
print(t.numpy())
print(t.shape)

2.7
()


### 1.1.6 strings

In [7]:
t = tf.constant("cafe")
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [8]:
t = tf.constant(["cafe", "coffee", "咖啡"])
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


### 1.1.7 ragged tensor

In [9]:
# index
r1 = tf.ragged.constant([[1, 2], [3, 4, 5], [], [41]])
print(r1)
print(r1[1:2])

<tf.RaggedTensor [[1, 2], [3, 4, 5], [], [41]]>
<tf.RaggedTensor [[3, 4, 5]]>


In [10]:
# operation
r2 = tf.ragged.constant([[5,6], [], [9]])  
r3 = tf.ragged.constant([[1, 2, 3], [2], [3, 4], [1]])
print(tf.concat([r1, r2], axis=0))
print(tf.concat([r1, r3], axis=1))

<tf.RaggedTensor [[1, 2], [3, 4, 5], [], [41], [5, 6], [], [9]]>
<tf.RaggedTensor [[1, 2, 1, 2, 3], [3, 4, 5, 2], [3, 4], [41, 1]]>


In [11]:
# convert to tensor
print(r1.to_tensor())
r4 = tf.ragged.constant([["abc", "bcd"], [], ["中国", "美国"]])
print(r4.to_tensor())    # 用零和空填补

tf.Tensor(
[[ 1  2  0]
 [ 3  4  5]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)
tf.Tensor(
[[b'abc' b'bcd']
 [b'' b'']
 [b'\xe4\xb8\xad\xe5\x9b\xbd' b'\xe7\xbe\x8e\xe5\x9b\xbd']], shape=(3, 2), dtype=string)


### 1.1.8 Sparsetensor

In [12]:
# 注意：tf.SparseTensor()中indices必须是排好序的，
# 否则要用tf.sparse.reorder函数重新排序
s1 = tf.SparseTensor(indices = [[0, 1], [1, 0], [2, 3]],
                     values = [1., 2., 3.],
                     dense_shape = [3, 4])
print(s1)
print(tf.sparse.to_dense(s1))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [13]:
# operation
s2 = s1 * 2.0
print(s2)
# 没有加法操作
try:
    s3 = s1 + 1.
except TypeError as ex:
    print(ex)

s4 = tf.constant([[10., 20.],
                  [30., 40.],
                  [50., 60.],
                  [70., 80.]])
print(tf.sparse.sparse_dense_matmul(s1, s4)) # 矩阵乘法

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'float'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [14]:
# 注意：tf.SparseTensor()中indices必须是排好序的，否则要用tf.sparse.reorder函数重新排序
s5 = tf.SparseTensor(indices = [[0, 2], [0, 1], [2, 3]],
                     values = [1., 2., 3.],
                     dense_shape = [3, 4])
print(s5)
s6 = tf.sparse.reorder(s5)
try:
    print(tf.sparse.to_dense(s5))
except:
    print("InvalidArgumentError")
print(tf.sparse.to_dense(s6))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
InvalidArgumentError
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


## 2. Variable

### 2.1 definition

In [15]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]],
                dtype = tf.float32)
print(v)
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


### 2.2 assign value

In [16]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]],
                dtype = tf.float32)

v.assign(2 * v)
print(v.numpy())

v[0, 1].assign(42)
print(v.numpy())

v[1].assign([7., 8., 9.])
print(v.numpy())

try:
    v[1] = [7., 8., 9.]    # 不能通过等号赋值
except TypeError as ex:
    print(ex)

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 7.  8.  9.]]
'ResourceVariable' object does not support item assignment
